# 크롤링

### 아마존 리뷰 데이터 수집

In [ ]:
import math
import traceback
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import urllib.parse
import json

# === 크롬 옵션 설정 ===
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("disable-popup-blocking")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# === 사용자가 직접 입력할 ASIN 리스트 ===
# 여기에 크롤링하고 싶은 30개 (또는 원하는 개수) 제품의 ASIN을 입력하세요.
ASINS_TO_CRAWL = [
    "B07MJL8NXR",
    "B08SRSRN7G", "B08FY6SCR8", "B0CK9QNYZW", "B07MFYYZ5B", "B0DRPSBQH3",
    "B07ZP4HBXS", "B07W6B8ZLZ", "B09VX8WLQQ", "B07ZP461TY", "B07MCCVZYC",
    "B0DDYDGCSD" , "B0015068TG", "B0DY1ZS88N", "B0F6F8TBR7", "B0DY21L16Y",
    "B0F3DNXDRZ", "B0791ZPVTY", "B0F3DRGDGM", "B09SGVCNFK", "B0791Y2LMB",
    "B09SGQVPDW", "B07NLQ5V6L", "B08WHYHPL5", "B08WHMBTM6", "B0DBRP7NN1",
    "B09NZ54BR4", "B0DBRP2KTM", "B0BV9749Q3", "B09NZPZ8Y6", "B0CPFX647Y",
    "B0CPFTBWYJ", "B005V9UG18", "B00AY6S2KU", "B003FDC2I2", "B0F3PZSNLQ",
    "B089ZVRR82", "B01KITQCW2", "B0DN83F6QS", "B09V4P2DVF", "B01KITQG0A",
    "B01KITQEPM", "B07ZZCM9SZ", "B0D6DKR58D", "B09G71BM3N", "B0854W9ND4",
    "B07ZZCDZST", "B0D7TFG59T", "B0B69YRPXH", "B08B2XCK56", "B0B75R8Z7X",
    "B01HOPJ2WU", "B07BL5ZHF9", "B07BL5BT2V", "B07BL5G7TC", "B0B6WRW3GX",
    "B01HOPJ8V0", "B01HOPJ3OC", "B0DS6MT455", "B0DS6LWMKJ", "B0DS6LGSV6",
    "B0D8MBS6XX", "B0D8M7RSHX", "B0D8M7K536", "B0DTMHYMG8", "B09V1S9168",
    "B0D823BK53", "B09V1S3V12", "B0DS6KMXHH", "B0DS6KYV4T", "B0DS6LSXXP",
    "B074P98SV7", "B07NH274NS", "B0C6YLVTHP", "B074P9D8JK", "B0DLCNXJD1",
    "B0DLBVHTB5", "B0DLCDBY96", "B0DHWD5WBM", "B0DHWDLW5X", "B0036F576I",
    "B0DQYV1Y51", "B06XWZHPN8", "B00166BBXW", "B0BSXXCD97", "B0034L4YGI",
    "B084WB9T61", "B0CFGBC9G2", "B01769TWGA", "B0036FBZPK", "B0CCWXBD5D",
    "B07BDS1YFX", "B085LSZCJ7", "B085LVFFKM", "B08B2J4DKZ", "B08TJ4LQ1N",
    "B08B7B3CF8", "B09LZZL1FF", "B0CTJ7G7N3", "B08GLMGVJQ", "B09B7Z7WGG",
    "B09PFK7H3W", "B09BC7V1JJ", "B08GLMT52Z", "B0BCSZPGQ3", "B079WNBTSH",
    "B0CM43QLM7", "B0B5M6RMFD", "B09BBWYD8X", "B0015068PA", "B0C5YVDRVM",
    "B09BBY5GPD", "B00BKL4KKY", "B0CDHNX333", "B00HKF9KFE", "B00BYOW7VG",
    "B0BKH8LN24", "B0BKH6MYD1", "B0CP8HZW65", "B0BKH4JXTM", "B01DDIRDZA",
    "B09GJVN2YZ", "B0DZY51RWP", "B0DZ8KV821", "B08QQKRWPR", "B08QQZ71P6",
    "B0BNFB7PSG", "B09NCJYC6K", "B09NCGMWX6", "B08YB4JQN4", "B08QQBGL95",
    "B08YS3TWTT"
]

# 모든 제품의 기본 리뷰 URL에 formatType=current_format 고정
# 리뷰 파일명에 사용될 Flavor 식별자.
FLAVOR_IDENTIFIER_FOR_FILENAME = "current_format_reviews"


def crawl_product_info_and_reviews_combined(asin, review_star_filters=None):
    if review_star_filters is None:
        review_star_filters = ["5", "4", "3", "2", "1"]  # 별점 필터 순서

    star_filter_map = {
        "5": "five_star",
        "4": "four_star",
        "3": "three_star",
        "2": "two_star",
        "1": "one_star",
    }

    product_data = {}
    product_url = f"https://www.amazon.com/dp/{asin}"

    print(f"\n--- ASIN: {asin} 제품 정보 및 리뷰 수집 시작 ---")

    driver.get(product_url)
    time.sleep(3)

    try:
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "productTitle")))
    except Exception as e:
        print(f"오류: 제품 {asin} 상세페이지 로딩 실패 또는 productTitle 요소를 찾을 수 없습니다: {e}")
        return None

    if "/pharmacy/" in driver.current_url:
        print(f"DEBUG: 제품 {asin} pharmacy 리디렉션 감지, 이 제품은 수집하지 않습니다.")
        return None

    # 제품명
    try:
        product_title = driver.find_element(By.ID, "productTitle").text.strip()
    except:
        product_title = "N/A"
        print(f"DEBUG: ASIN {asin} 제품명 추출 실패")

    # 대표 이미지 URL
    try:
        img_url = driver.find_element(By.ID, "landingImage").get_attribute("src")
    except:
        img_url = "N/A"
        print(f"DEBUG: ASIN {asin} 이미지 URL 추출 실패")

    # 가격
    try:
        price_whole = driver.find_element(By.CSS_SELECTOR, "span.a-price-whole").text.strip()
        price_fraction = driver.find_element(By.CSS_SELECTOR, "span.a-price-fraction").text.strip()
        price_whole_clean = price_whole.replace(",", "")
        price = f"${price_whole_clean}.{price_fraction}"
        try:
            price_per_count = driver.find_element(By.CSS_SELECTOR, "span.a-price a-price-symbol + span").text.strip()
            price += f" ({price_per_count})"
        except:
            pass
    except:
        price = "N/A"
        print(f"DEBUG: ASIN {asin} 가격 추출 실패")

    # Product Overview
    product_overview = {}
    try:
        overview_table = driver.find_element(By.ID, "productOverview_feature_div")
        rows = overview_table.find_elements(By.CSS_SELECTOR, "tr")
        for row in rows:
            try:
                th = row.find_element(By.TAG_NAME, "th").text.strip()
                td = row.find_element(By.TAG_NAME, "td").text.strip()
                product_overview[th] = td
            except:
                continue
    except:
        print(f"DEBUG: ASIN {asin} Product Overview 추출 실패")

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)

    # Product Description
    product_description = ""
    try:
        desc_div = driver.find_element(By.ID, "productDescription")
        product_description = desc_div.text.strip()
    except:
        pass
    if not product_description:
        try:
            alt_desc = driver.find_element(By.CSS_SELECTOR, "div#feature-bullets ~ div.a-section.a-spacing-small.a-spacing-top-small")
            product_description = alt_desc.text.strip()
        except:
            print(f"DEBUG: ASIN {asin} Product Description 추출 실패")
            pass

    # About this item
    about_items = []
    try:
        about_div = driver.find_element(By.ID, "feature-bullets")
        bullet_points = about_div.find_elements(By.CSS_SELECTOR, "span.a-list-item")
        for bp in bullet_points:
            txt = bp.text.strip()
            if txt:
                about_items.append(txt)
    except:
        try:
            about_div = driver.find_element(By.ID, "feature-bullets_feature_div")
            bullet_points = about_div.find_elements(By.CSS_SELECTOR, "ul li")
            for bp in bullet_points:
                txt = bp.text.strip()
                if txt:
                    about_items.append(txt)
        except:
            print(f"DEBUG: ASIN {asin} About this item 추출 실패")
            pass

    # Important Information
    important_info = {}
    try:
        important_div = None
        try:
            important_div = driver.find_element(By.ID, "important-information")
        except:
            important_div = driver.find_element(By.ID, "importantInformation_feature_div")

        if important_div:
            sections = important_div.find_elements(By.CSS_SELECTOR, "div.a-section")
            for section in sections:
                try:
                    heading = section.find_element(By.TAG_NAME, "h3").text.strip()
                    content = section.find_element(By.TAG_NAME, "p").text.strip()
                    important_info[heading] = content
                except:
                    text = section.text.strip()
                    if text:
                        important_info[f"section_{len(important_info)+1}"] = text
    except:
        print(f"DEBUG: ASIN {asin} Important Information 추출 실패")
        pass

    # 제품 정보 딕셔너리 구성
    product_data = {
        "asin": asin,
        "title": product_title,
        "image_url": img_url,
        "price": price,
        "product_overview": json.dumps(product_overview, ensure_ascii=False), # JSON 문자열로 저장
        "product_description": product_description,
        "about_this_item": json.dumps(about_items, ensure_ascii=False),     # JSON 문자열로 저장
        "important_information": json.dumps(important_info, ensure_ascii=False), # JSON 문자열로 저장
        "product_url": product_url,
    }

    print(f"✅ ASIN {asin} 제품 정보 수집 완료.")

    # 리뷰 크롤링 시작
    all_reviews_raw = [] # 모든 별점의 리뷰 데이터 ( Flavor, Reviewer, Rating, Title, Body )

    # 모든 제품의 기본 리뷰 URL에 formatType=current_format 고정
    base_review_url = f"https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_fmt?ie=UTF8&reviewerType=all_reviews&formatType=current_format"


    for star_filter in review_star_filters:
        reviews_this_star = []

        filter_param = star_filter_map.get(star_filter, None)
        if not filter_param:
            continue

        review_url_with_stars = f"{base_review_url}&filterByStar={filter_param}"

        driver.get(review_url_with_stars)
        time.sleep(3)

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span[data-hook='review-body']"))
            )
        except Exception as e:
            print(f"❌ {star_filter}⭐ 리뷰 초기 로드 실패 for ASIN {asin} (오류: {e})")
            continue

        while True:
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)

                review_blocks = driver.find_elements(By.CSS_SELECTOR, "li[data-hook='review']")

                for review_block in review_blocks:
                    try:
                        rating_text = review_block.find_element(By.CSS_SELECTOR, "i[data-hook='review-star-rating'] span.a-icon-alt").text.strip()
                    except:
                        rating_text = "N/A"

                    title = "N/A"
                    translated_title_text = ""
                    original_title_text = ""

                    try:
                        translated_title_elem = review_block.find_element(By.CSS_SELECTOR, "a[data-hook='review-title'] .cr-translated-review-content")
                        translated_title_text = translated_title_elem.get_attribute('textContent').strip() if translated_title_elem else ""
                    except:
                        pass

                    try:
                        original_title_elem = review_block.find_element(By.CSS_SELECTOR, "a[data-hook='review-title']")
                        original_title_text = original_title_elem.text.strip()
                    except:
                        pass

                    if translated_title_text:
                        title = translated_title_text
                    elif original_title_text:
                        title = original_title_text


                    try:
                        reviewer = review_block.find_element(By.CSS_SELECTOR, "span.a-profile-name").text.strip()
                    except:
                        reviewer = "N/A"

                    body = "N/A"
                    translated_body_text = ""
                    original_body_text = ""
                    try:
                        translated_body_elem = review_block.find_element(By.CSS_SELECTOR, "span.cr-translated-review-content")
                        translated_body_text = translated_body_elem.get_attribute('textContent').strip() if translated_body_elem else ""
                    except:
                        pass

                    try:
                        original_body_elem = review_block.find_element(By.CSS_SELECTOR, "span[data-hook='review-body']")
                        original_body_text = original_body_elem.text.strip()
                    except:
                        pass

                    if translated_body_text:
                        body = translated_body_text
                    elif original_body_text:
                        body = original_body_text


                    try:
                        format_strip = review_block.find_element(By.CSS_SELECTOR, "a[data-hook='format-strip']").text
                        flavor = "N/A"
                        for segment in format_strip.split("|"):
                            if "Flavor Name" in segment:
                                flavor = segment.strip().replace("Flavor Name:", "").strip()
                                break
                    except:
                        flavor = "N/A"

                    review_data_row = {
                        "Flavor": flavor,
                        "Reviewer": reviewer,
                        "Rating": rating_text,
                        "Title": title,
                        "Body": body
                    }
                    if review_data_row not in reviews_this_star:
                        reviews_this_star.append(review_data_row)

                try:
                    next_btn = driver.find_element(By.CSS_SELECTOR, "li.a-last a")
                    next_btn.click()
                    time.sleep(3)
                except:
                    break

            except Exception as e:
                print(f"오류: 리뷰 페이지 처리 중 오류 발생 (ASIN: {asin}, 별점: {star_filter}⭐): {e}")
                traceback.print_exc()
                break

        print(f"✅ ASIN {asin}의 'Current Format'에 대한 {star_filter}⭐ 리뷰 {len(reviews_this_star)}개 수집됨")
        all_reviews_raw += reviews_this_star

    print(f"✅ ASIN {asin}의 총 {len(all_reviews_raw)}건의 리뷰 수집 완료.")

    # --- 제품 정보와 리뷰를 하나의 CSV 파일로 저장 ---
    combined_filename = f"amazon_product_and_reviews_combined_{asin}_{FLAVOR_IDENTIFIER_FOR_FILENAME}.csv"

    # CSV 헤더 정의
    # 제품 정보 필드 (리뷰와 겹치지 않게 'Product_' 접두사 사용)
    product_fields = [f"Product_{k}" for k in product_data.keys() if k not in ["asin"]]
    # 리뷰 필드 (기존 이름 사용)
    review_fields = ["Flavor", "Reviewer", "Rating", "Title", "Body"]

    # ASIN은 모든 행에 공통으로 들어갈 것이므로 따로 정의
    fieldnames = ["ASIN"] + product_fields + review_fields

    with open(combined_filename, "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        # 첫 번째 행에 제품 정보와 첫 번째 리뷰를 함께 저장 (리뷰가 있다면)
        # 리뷰가 없으면 제품 정보만 저장
        first_row_data = {"ASIN": asin}

        # 제품 정보 추가
        for k, v in product_data.items():
            if k != "asin": # ASIN은 이미 위에서 추가했으므로 제외
                first_row_data[f"Product_{k}"] = v

        if all_reviews_raw:
            # 첫 번째 리뷰 데이터 추가
            first_review = all_reviews_raw[0]
            for k, v in first_review.items():
                first_row_data[k] = v
            writer.writerow(first_row_data)

            # 나머지 리뷰 데이터 추가 (제품 정보는 비워두고 리뷰 데이터만 채움)
            for review_row in all_reviews_raw[1:]:
                row_to_write = {"ASIN": asin} # ASIN은 계속 포함
                for k, v in review_row.items():
                    row_to_write[k] = v
                writer.writerow(row_to_write)
        else:
            # 리뷰가 없는 경우 제품 정보만 첫 행에 저장
            writer.writerow(first_row_data)

    print(f"✅ 제품 {asin} 정보 및 리뷰 총 {len(all_reviews_raw) + (1 if product_data else 0)}건 통합 저장 완료: {combined_filename}")

    return product_data # 이 함수는 더 이상 제품 정보만 반환하지 않지만, 호출 구조 유지를 위해 product_data 반환.

# === 메인 실행 로직 ===
driver.get("https://www.amazon.com/")
input("Amazon 웹페이지에서 로그인 완료 후, 콘솔에 Enter 키를 눌러주세요...")

print(f"\n--- {len(ASINS_TO_CRAWL)}개의 ASIN에 대한 제품 정보 및 리뷰 크롤링 시작 ---")

MAX_RETRIES = 3

for i, asin in enumerate(ASINS_TO_CRAWL):
    print(f"\n[{i+1}/{len(ASINS_TO_CRAWL)}] ASIN: {asin} 크롤링 중...")

    retries = 0
    while retries < MAX_RETRIES:
        try:
            # 함수 이름 변경: crawl_product_info_and_reviews_combined
            result = crawl_product_info_and_reviews_combined(asin)
            if result:
                print(f"✅ ASIN {asin} 크롤링 성공.")
                break
            else:
                print(f"⚠️ ASIN {asin} 제품 정보를 가져오지 못했습니다. 재시도합니다. (시도 {retries+1}/{MAX_RETRIES})")
                retries += 1
                time.sleep(5)
        except Exception as e:
            print(f"❌ ASIN {asin} 크롤링 중 예외 발생: {e}")
            traceback.print_exc()
            retries += 1
            if retries < MAX_RETRIES:
                print(f"재시도 중... (시도 {retries}/{MAX_RETRIES})")
                time.sleep(10)
            else:
                print(f"❌ ASIN {asin} 크롤링 {MAX_RETRIES}회 재시도 실패. 이 ASIN은 건너킵니다.")

    if retries == MAX_RETRIES and not result:
        print(f"⚠️ ASIN {asin} 크롤링 최종 실패. 건너킵니다.")

print("\n--- 모든 제품 정보 및 리뷰 크롤링이 완료되었습니다. ---")

driver.quit()

### 네이버 리뷰 데이터 수집

In [ ]:
options = webdriver.ChromeOptions()  # 크롬 옵션 객체 생성
options.add_argument("window-size-1920x1080")  # 전체화면
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")

In [ ]:
chrome_driver_path = "driver_path"
service = Service(executable_path = chrome_driver_path)

In [ ]:
driver = webdriver.Chrome(service = service, options = options)
driver.implicitly_wait(3)

In [ ]:
driver.get("URL_PATH")
time.sleep(3)

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)

In [ ]:
driver.find_element(By.CSS_SELECTOR, "#content > div > div._3CTsMZymJs > div:nth-child(3) > div._27jmWaPaKy > ul > li:nth-child(2) > a").click()
time.sleep(5)

In [ ]:
driver.find_element(By.CSS_SELECTOR, "#REVIEW > div > div._2LvIMaBiIO > div._3aC7jlfVdk > div._1txuie7UTH > ul > li:nth-child(2) > a").click()
time.sleep(3)

In [ ]:
# 현재 페이지
page_num = 1
page_ctl = 3

write_dt_lst = []
item_nm_lst = []
content_lst = []

# 날짜
date_cut = (datetime.now() - timedelta(days=365)).strftime('%Y%m%d')

while True:
    if page_num == 26:
        print("500개 수집 완료")
        break

    print(f'start : {page_num} page 수집 중, page_ctl:{page_ctl}')

    # 1. 셀레니움으로 html 가져오기
    html_source = driver.page_source

    # 2. bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 3. 리뷰 정보 가져오기
    reviews = soup.findAll('li', {'class': 'BnwL_cs1av'})

    # 4. 한 페이지 내에서 수집 가능한 리뷰 리스트에 저장
    for review in range(len(reviews)):
        try:
            # 4-1. 리뷰 작성일자 수집
            write_dt_raw = reviews[review].findAll('span', {'class': '_2L3vDiadT9'})[0].get_text()
            write_dt = datetime.strptime(write_dt_raw, '%y.%m.%d.').strftime('%Y%m%d')
        except Exception:
            write_dt = ''

        # 4-2. 상품명 수집
        try:
            item_nm_divs = reviews[review].findAll('div', {'class': '_2FXNMst_ak'})
            if item_nm_divs:
                item_nm_info_raw = item_nm_divs[0].get_text()

                # dl 태그 안의 텍스트 추출 (없을 수도 있음)
                dl_tag = item_nm_divs[0].find('dl', {'class': 'XbGQRlzveO'})
                item_nm_info_for_del = dl_tag.get_text() if dl_tag else ''

                # 텍스트 정제
                item_nm_info = re.sub(item_nm_info_for_del, '', item_nm_info_raw)

                # '제품 선택: ' 위치 찾기
                str_start_idx = item_nm_info.find('제품 선택: ')
                if str_start_idx != -1:
                    item_nm = item_nm_info[str_start_idx + len('제품 선택: '):].strip()
                else:
                    item_nm = item_nm_info.strip()
            else:
                item_nm = ''
        except Exception:
            item_nm = ''

        # 4-3. 리뷰내용 수집
        try:
            review_div = reviews[review].findAll('div', {'class': '_1kMfD5ErZ6'})
            if review_div:
                span_tag = review_div[0].find('span', {'class': '_2L3vDiadT9'})
                if span_tag:
                    review_content_raw = span_tag.get_text()
                    review_content = re.sub(' +', ' ', re.sub('\n', ' ', review_content_raw))
                else:
                    review_content = ''
            else:
                review_content = ''
        except Exception:
            review_content = ''

        # 4-4. 수집데이터 저장
        write_dt_lst.append(write_dt)
        item_nm_lst.append(item_nm)
        content_lst.append(review_content)

    # 5. 리뷰 수집일자 기준 데이터 확인 (최근 1년치만 수집)
    if write_dt_lst and write_dt_lst[-1] < date_cut:
        break

    # 6. 페이지 이동
    try:
        driver.find_element(By.CSS_SELECTOR, f'#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a:nth-child({page_ctl})').click()
        time.sleep(5)
    except Exception as e:
        print(f"페이지 이동 실패: {e}")
        break

    page_num += 1
    page_ctl += 1
    if page_num % 10 == 1:
        page_ctl = 3

print('done')

In [ ]:
result_df = pd.DataFrame({
              'Item_info' : item_nm_lst,
              'Content' : content_lst,
              'Date' : write_dt_lst })

### 옥션 리뷰 데이터 수집

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

# 🔽 복사한 <ul class="list-item"> 전체 HTML을 여기에 붙여넣기
html = """

"""

# 🌿 파싱 시작
soup = BeautifulSoup(html, "html.parser")
review_items = soup.select("li.list-item")

data = []

for item in review_items:
    review_text = item.select_one('.box__review-text .text')
    writer = item.select_one('.text__writer')
    date = item.select_one('.text__date')
    rating = item.select_one('.sprite__vip.image__star .for-a11y')
    rating_value = ''

    if rating:
        text = rating.get_text(strip=True)
        # 예: '이용자 평점 4점' → 숫자만 추출
        import re
        match = re.search(r'(\d(?:\.\d)?)점', text)
        if match:
            rating_value = match.group(1)

    data.append({
        '제품명': '제품명',
        '상품번호':'',
        '리뷰': review_text.get_text(strip=True) if review_text else '',
        '작성자': writer.get_text(strip=True) if writer else '',
        '날짜': date.get_text(strip=True) if date else '',
        '별점': rating_value
    })


# 🧾 CSV로 저장
df = pd.DataFrame(data)
df.to_csv("제품명.csv", index=False, encoding="utf-8-sig")
print(f"✅ 저장 완료: 11st_reviews_ul.csv (총 {len(data)}개의 리뷰가 모였습니다.)")


# 데이터 분석

### 워드 클라우드

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from mecab import MeCab # 'mecab-python3' 라이브러리에서 직접 MeCab을 불러옵니다.
import re
import os
from collections import Counter # 단어 빈도수를 세기 위해 Counter를 추가했습니다.

# matplotlib 한글 폰트 설정
# 폰트 경로가 정확한지 확인해 주세요.
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

# Mecab 초기화 (사전 경로를 자동으로 찾아줍니다.)
try:
    mecab = MeCab()
    print("MeCab이 성공적으로 초기화되었습니다.")
except Exception as e:
    print(f"MeCab 초기화 중 오류 발생: {e}")
    # MeCab 초기화 실패 시 스크립트를 종료합니다.
    exit()

# --- 2. 함수 정의 ---
# 텍스트에서 명사를 추출하고 불용어를 제거하는 함수
def get_nouns(text, stop_words):
    text = str(text) # 입력값을 문자열로 변환하여 에러 방지
    # 한글, 영어, 숫자만 남기고 특수문자 제거
    text = re.sub('[^가-힣a-zA-Z0-9\s]', '', text)

    # 명사만 추출
    nouns = mecab.nouns(text)

    # 한 글자 명사 및 불용어 제거
    # `stop_words` 리스트를 외부에서 받아서 더 유연하게 관리합니다.
    filtered_nouns = [n for n in nouns if len(n) > 1 and n not in stop_words]

    return filtered_nouns

# --- 3. 메인 로직 실행 ---
def main():
    # CSV 파일 불러오기 (실제 파일 경로로 변경)
    file_path = '/content/drive/MyDrive/데이터 분석 및 시각화/1. 키워드 추출/protein_1.csv'

    try:
        df = pd.read_csv(file_path)
        print("CSV 파일이 성공적으로 로드되었습니다.")
    except FileNotFoundError:
        print(f"오류: {file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
        return # 파일이 없으면 함수 종료

    # 네가 추가했던 불용어 리스트를 따로 정의
    stop_words = ['정도', '때문', '부분', '생각', '제품', '것', '저', '다', '배송']

    # '리뷰' 컬럼의 모든 리뷰 텍스트를 하나의 문자열로 합칩니다.
    # 결측치(NaN)는 제외하고 합쳐줍니다.
    all_reviews = ' '.join(df['리뷰'].dropna().astype(str))

    # 텍스트 데이터가 비어 있는지 확인
    if not all_reviews.strip():
        print("오류: '리뷰' 컬럼에 유효한 텍스트 데이터가 없어 워드 클라우드를 생성할 수 없습니다.")
        return

    # 명사 추출
    nouns = get_nouns(all_reviews, stop_words)

    # 추출된 명사 리스트가 비어 있는지 확인 (흰 바탕 문제 방지)
    if not nouns:
        print("오류: 명사 추출 결과가 비어 있습니다. 워드 클라우드를 생성할 수 없습니다.")
        return

    # 명사 리스트의 빈도수를 계산하여 딕셔너리 형태로 변환
    word_counts = Counter(nouns)

    # 워드 클라우드 생성
    wordcloud = WordCloud(
        font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
        width=800,
        height=400,
        max_words=50,
        background_color='white'
    )
    # 빈도수 딕셔너리를 사용하여 워드 클라우드 생성
    wordcloud.generate_from_frequencies(word_counts)

    # 시각화
    plt.figure(figsize=(10, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

    # 이미지 저장
    # 저장할 경로와 파일명을 설정해 주세요.
    save_path = '/content/drive/MyDrive/데이터 분석 및 시각화/1. 키워드 추출/image/protein_image.png'
    # wordcloud 객체의 to_file() 함수를 사용하여 이미지를 저장합니다.
    wordcloud.to_file(save_path)
    print(f"이미지가 성공적으로 저장되었습니다: {save_path}")

# 이 코드를 실행합니다.
if __name__ == "__main__":
    main()

### 감성 분석 및 문서 요약

In [ ]:
import pandas as pd
import os

def split_reviews_by_language(file_path, english_start_row, english_end_row):
    """
    전체 리뷰 데이터를 받아 영어와 한글 리뷰로 분리하여 CSV 파일로 저장하는 함수.

    Args:
        file_path (str): 전체 리뷰 데이터가 담긴 CSV 파일 경로.
        english_start_row (int): 영어 리뷰의 시작 행 번호 (1부터 시작).
        english_end_row (int): 영어 리뷰의 끝 행 번호 (1부터 시작).
    """
    try:
        # 1. 전체 데이터를 불러옵니다.
        df = pd.read_csv(file_path)
        print(f"✔️ 전체 데이터 로드 완료: {len(df)}개 행")

        # 2. 영어 리뷰를 슬라이싱합니다.
        # pandas는 0부터 시작하므로 행 번호에서 1을 빼서 인덱스에 맞춥니다.
        english_df = df.iloc[english_start_row - 1 : english_end_row]
        print(f"✔️ 영어 리뷰 추출 완료: {len(english_df)}개 행")

        # 3. 한글 리뷰를 슬라이싱합니다. (영어 리뷰를 제외한 나머지)
        korean_df = df.drop(index=range(english_start_row - 1, english_end_row))
        print(f"✔️ 한글 리뷰 추출 완료: {len(korean_df)}개 행")

        # 4. 각 DataFrame을 새로운 CSV 파일로 저장할 경로를 설정합니다.
        output_folder = '/content/drive/MyDrive/데이터 분석 및 시각화/1. 키워드 추출'
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print(f"'{output_folder}' 폴더가 생성되었습니다.")

        # 5. 각 DataFrame을 새로운 CSV 파일로 저장합니다.
        english_file_path = os.path.join(output_folder, '영어_리뷰_데이터.csv')
        korean_file_path = os.path.join(output_folder, '한글_리뷰_데이터.csv')

        english_df.to_csv(english_file_path, index=False)
        korean_df.to_csv(korean_file_path, index=False)

        print("\n🎉 모든 작업이 완료되었습니다!")
        print(f" '영어_리뷰_데이터.csv' 파일과 '한글_리뷰_데이터.csv' 파일이 '{output_folder}'에 저장되었습니다.")

    except FileNotFoundError:
        print(f"오류: {file_path} 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

# 실행 코드
file_path = '/content/drive/MyDrive/데이터 수집/3. 전처리X 데이터 통합/통합데이터(간단한_텍스트_전처리).csv'
english_start_row = 67849
english_end_row = 130700

split_reviews_by_language(file_path, english_start_row, english_end_row)

In [ ]:
import pandas as pd
import os

# 1. 파일 경로를 지정해 주세요.
file_path = '/content/drive/MyDrive/데이터 분석 및 시각화/1. 키워드 추출/영어_리뷰_데이터.csv'

try:
    df = pd.read_csv(file_path)

    # 2. 총 행 개수와 중간점 계산
    total_rows = len(df)
    split_point = total_rows // 2

    print(f"✔️ 데이터의 총 행 개수: {total_rows}")
    print(f"✔️ 데이터를 나눌 중간점: {split_point}")

    # 3. 데이터프레임을 두 개로 분할
    df_part1 = df.iloc[:split_point].copy()
    df_part2 = df.iloc[split_point:].copy()

    # 4. 원하는 경로와 파일 이름으로 각각 저장하기
    output_folder = '/content/drive/MyDrive/데이터 분석 및 시각화/1. 키워드 추출'

    # 폴더가 없으면 새로 생성
    os.makedirs(output_folder, exist_ok=True)

    # 첫 번째 파일 경로 설정 및 저장
    output_path1 = os.path.join(output_folder, '영어_리뷰_데이터_part1.csv')
    df_part1.to_csv(output_path1, index=False, encoding='utf-8-sig')

    # 두 번째 파일 경로 설정 및 저장
    output_path2 = os.path.join(output_folder, '영어_리뷰_데이터_part2.csv')
    df_part2.to_csv(output_path2, index=False, encoding='utf-8-sig')

    print(f"\n🎉 파일이 아래 두 경로에 성공적으로 저장되었습니다!")
    print(f"- 첫 번째 파일: '{output_path1}'")
    print(f"- 두 번째 파일: '{output_path2}'")

except FileNotFoundError:
    print(f"오류: '{file_path}' 파일을 찾을 수 없습니다. 경로를 확인해주세요.")

##### 한글 리뷰

In [ ]:

def chunk_text(reviews_list, max_tokens=500):
    """리뷰 리스트를 받아 최대 토큰 수에 맞게 텍스트 덩어리로 나누는 함수"""
    text_to_chunk = " ".join(reviews_list)
    chunks = []
    current_chunk = ""
    current_tokens = 0
    sentences = text_to_chunk.split('.')
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence: continue
        sentence_tokens = len(sentence.split())
        if current_tokens + sentence_tokens > max_tokens:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
            current_tokens = sentence_tokens
        else:
            current_chunk += sentence + ". "
            current_tokens += sentence_tokens
    if current_chunk: chunks.append(current_chunk.strip())
    return chunks

def summarize_korean(text, summarizer):
    """한글 리뷰만 요약하는 함수"""
    try:
        return summarizer(text, max_length=150, min_length=30)[0]['summary_text']
    except Exception as e:
        return f"요약 실패: {e}"

def main():
    print("--- Hugging Face 기반 한글 리뷰 카테고리별 분석 시작 ---")

    # 1. 한글 리뷰 데이터 파일을 불러옵니다. (경로를 확인해 주세요)
    file_path = '/content/drive/MyDrive/데이터 분석 및 시각화/1. 키워드 추출/한글_리뷰_데이터.csv'
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"오류: {file_path} 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        return

    # 2. 분석할 카테고리 리스트를 정의합니다.
    categories = ['단백질', '비타민', '아연', '오메가3', '유산균', '단백질 쉐이크', '멀티비타민']

    # 3. 감성 분석 모델을 로드합니다.
    print("\n[1단계] 감성 분석 모델을 로드하고 있습니다...")
    try:
        classifier = pipeline(
            "sentiment-analysis",
            model="cardiffnlp/twitter-xlm-roberta-base-sentiment",
            tokenizer="cardiffnlp/twitter-xlm-roberta-base-sentiment",
            truncation=True,
            max_length=500
        )
    except Exception as e:
        print(f"감성 분석 모델 로드 중 오류가 발생했습니다: {e}")
        return

    # 4. 한글 요약 모델을 로드합니다.
    print("[2단계] 한글 요약 모델을 로드하고 있습니다...")
    try:
        korean_summarizer = pipeline(
            "summarization",
            model="lcw99/t5-base-korean-text-summary"
        )
    except Exception as e:
        print(f"한글 요약 모델 로드 중 오류가 발생했습니다: {e}")
        return

    # 요약 결과를 저장할 리스트를 초기화합니다.
    summary_results = []

    # 5. 카테고리별 분석 루프 시작
    for category in categories:
        print(f"\n--- {category} 카테고리 분석 시작 ---")

        # 5-1. 해당 카테고리의 리뷰만 추출합니다.
        category_df = df[df['카테고리'] == category].copy()

        if category_df.empty:
            print("해당 카테고리의 리뷰가 없습니다. 다음 카테고리로 넘어갑니다.")
            continue

        # 5-2. 감성 분석을 진행합니다.
        print("감성 분석 중...")
        category_df['sentiment'] = category_df['리뷰'].apply(
            lambda review: classifier(review)[0]['label'] if isinstance(review, str) else 'neutral'
        )
        print("분석 완료.")

        # 5-3. 긍정/부정 리뷰를 분리합니다.
        positive_reviews = category_df[category_df['sentiment'] == 'positive']['리뷰'].tolist()
        negative_reviews = category_df[category_df['sentiment'] == 'negative']['리뷰'].tolist()

        # 5-4. 긍정 리뷰 요약
        if positive_reviews:
            print(f"✔️ 긍정 리뷰 ({len(positive_reviews)}개) 요약")
            positive_chunks = chunk_text(positive_reviews)
            for i, chunk in enumerate(positive_chunks):
                summary = summarize_korean(chunk, korean_summarizer)
                # 요약 결과를 리스트에 추가합니다.
                summary_results.append({
                    'category': category,
                    'sentiment': 'positive',
                    'summary': summary
                })

        # 5-5. 부정 리뷰 요약
        if negative_reviews:
            print(f"✔️ 부정 리뷰 ({len(negative_reviews)}개) 요약")
            negative_chunks = chunk_text(negative_reviews)
            for i, chunk in enumerate(negative_chunks):
                summary = summarize_korean(chunk, korean_summarizer)
                # 요약 결과를 리스트에 추가합니다.
                summary_results.append({
                    'category': category,
                    'sentiment': 'negative',
                    'summary': summary
                })

    # 6. 요약 결과를 CSV 파일로 저장합니다.
    print("\n--- 분석 결과를 CSV 파일로 저장합니다. ---")
    output_folder = '/content/drive/MyDrive/데이터 분석 및 시각화/1. 키워드 추출/리뷰 결과'
    output_file = '한글_리뷰_요약_결과.csv'
    output_path = os.path.join(output_folder, output_file)

    if summary_results:
        summary_df = pd.DataFrame(summary_results)
        summary_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"🎉 요약 결과가 '{output_path}'에 성공적으로 저장되었습니다!")
    else:
        print("저장할 요약 결과가 없습니다.")


if __name__ == "__main__":
    # 필요한 라이브러리가 없다면 아래 코드를 실행해 설치하세요.
    # !pip install transformers pandas torch
    main()


##### 영어 리뷰

In [ ]:
def chunk_text(reviews_list, max_tokens=300):
    """리뷰 리스트를 받아 최대 토큰 수에 맞게 텍스트 덩어리로 나누는 함수"""
    text_to_chunk = " ".join(reviews_list)
    chunks = []
    current_chunk = ""
    current_tokens = 0
    # 문장 분리를 위한 정규식
    sentences = re.split(r'(?<!\.\w)(?<![A-Z][a-z])(?<=\.|\?|\!)\s', text_to_chunk)
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence: continue
        sentence_tokens = len(sentence.split())
        if current_tokens + sentence_tokens > max_tokens:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
            current_tokens = sentence_tokens
        else:
            current_chunk += sentence + " "
            current_tokens += sentence_tokens
    if current_chunk: chunks.append(current_chunk.strip())
    return chunks

def summarize_english(text, summarizer):
    """영어 리뷰만 요약하는 함수"""
    try:
        # 모델의 최대 입력 길이를 고려해 텍스트를 자릅니다.
        max_model_length = 512
        if len(text.split()) > max_model_length:
            text = " ".join(text.split()[:max_model_length])

        return summarizer(text, max_length=150, min_length=30)[0]['summary_text']
    except Exception as e:
        return f"요약 실패: {e}"

def main():
    print("--- Hugging Face 기반 고유번호 & 카테고리별 감성 분석 및 요약 ---")

    # 1. 영어 리뷰 데이터 파일을 불러옵니다. (경로를 확인해 주세요)
    file_path = '/content/drive/MyDrive/데이터 분석 및 시각화/1. 키워드 추출/영어_리뷰_데이터_part1.csv'
    try:
        df = pd.read_csv(file_path)
        # 필요한 열이 있는지 확인
        if '고유번호' not in df.columns or '리뷰' not in df.columns or '카테고리' not in df.columns:
            print("오류: 데이터프레임에 '고유번호', '카테고리' 또는 '리뷰' 열이 없습니다. 열 이름을 확인해 주세요.")
            return
    except FileNotFoundError:
        print(f"오류: {file_path} 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        return

    # 리뷰 열의 빈 값을 모두 빈 문자열로 채워줍니다.
    df['리뷰'] = df['리뷰'].fillna('')

    # 리뷰 열의 모든 값을 문자열 타입으로 변환합니다.
    df['리뷰'] = df['리뷰'].astype(str)

    # 2. 감성 분석 모델을 로드합니다.
    print("\n[1단계] 감성 분석 모델을 로드하고 있습니다...")
    try:
        classifier = pipeline(
            "sentiment-analysis",
            model="AnkitAI/reviews-roberta-base-sentiment-analysis",
            tokenizer="AnkitAI/reviews-roberta-base-sentiment-analysis",
            truncation=True,
            max_length=512
        )
    except Exception as e:
        print(f"감성 분석 모델 로드 중 오류가 발생했습니다: {e}")
        return

    # 3. 영어 요약 모델을 로드합니다.
    print("[2단계] 영어 요약 모델을 로드하고 있습니다...")
    try:
        english_summarizer = pipeline(
            "summarization",
            model="mabrouk/amazon-review-summarizer-bart"
        )
    except Exception as e:
        print(f"영어 요약 모델 로드 중 오류가 발생했습니다: {e}")
        return

    # 4. 리뷰에 감성 분석 결과를 추가합니다.
    print("\n[3단계] 리뷰 감성을 분석하고 있습니다...")
    # 리뷰 데이터를 리스트로 변환
    reviews_list = df['리뷰'].tolist()

    # 리스트 전체를 한 번에 모델에 전달
    results = classifier(reviews_list)

    # 결과만 데이터프레임에 추가 (모델이 직접 중립을 분류하므로 로직 단순화)
    def get_sentiment(result):
        # 'AnkitAI' 모델은 결과 딕셔너리를 하나만 반환합니다.
        label = result['label']

        # 모델 라벨(POSITIVE, NEGATIVE)을 그대로 사용
        return label

    # 새로 정의한 함수를 사용하여 감성 열을 채웁니다.
    # 결과는 [{'label': 'POSITIVE', 'score': 0.99}, ...]와 같은 형태입니다.
    df['sentiment'] = [get_sentiment(result) for result in results]
    print("분석 완료.")

    # 5. 고유번호, 카테고리, 감성별로 그룹화하고 요약합니다.
    print("\n[4단계] 고유번호, 카테고리, 감성별로 리뷰를 요약하고 있습니다...")

    # '고유번호', '카테고리', 'sentiment'를 기준으로 데이터를 그룹화합니다.
    grouped_df = df.groupby(['고유번호', '카테고리', 'sentiment'])['리뷰'].apply(lambda x: ' '.join(x)).reset_index()

    # 요약 결과를 저장할 리스트를 초기화합니다.
    summary_results = []

    for index, row in grouped_df.iterrows():
        unique_id = row['고유번호']
        category = row['카테고리']
        sentiment = row['sentiment']
        reviews_text = row['리뷰']

        print(f"--- 고유번호: {unique_id}, 카테고리: {category}, 감성: {sentiment} 요약 시작 ---")

        # 텍스트를 청크로 나누고 요약
        chunks = chunk_text([reviews_text])
        for chunk in chunks:
            summary = summarize_english(chunk, english_summarizer)
            summary_results.append({
                '고유번호': unique_id,
                '카테고리': category,
                'sentiment': sentiment,
                'summary': summary
            })

    # 6. 요약 결과를 CSV 파일로 저장합니다.
    print("\n--- 분석 결과를 CSV 파일로 저장합니다. ---")
    output_folder = '/content/drive/MyDrive/데이터 분석 및 시각화/1. 키워드 추출/리뷰 결과'
    output_file = '영어리뷰_감성_요약_결과_part1.csv'
    output_path = os.path.join(output_folder, output_file)

    if summary_results:
        summary_df = pd.DataFrame(summary_results)
        # 폴더가 없으면 생성
        os.makedirs(output_folder, exist_ok=True)
        summary_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"🎉 요약 결과가 '{output_path}'에 성공적으로 저장되었습니다!")
    else:
        print("저장할 요약 결과가 없습니다.")


if __name__ == "__main__":
    # 필요한 라이브러리가 없다면 아래 코드를 실행해 설치하세요.
    # !pip install transformers pandas torch
    main()

### GCP gemini API 이용한 리뷰요약

In [ ]:
# llm (genai)
!pip install llama-index-llms-google-genai llama-index

# 임베딩 (genai)
!pip install llama-index-embeddings-google-genai

!pip install --upgrade google-generativeai

### 필요한 함수 임폴트
import os
from dotenv import load_dotenv
# from llama_index.llms.openai import OpenAI
from llama_index.llms.google_genai import GoogleGenAI
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, Document, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
import tqdm
import pandas as pd

### df_grouped 데이터 프레임 임폴트

import pandas as pd

file_path = '/content/drive/MyDrive/임베딩/GCP gemini API 활용/1. 리뷰통합.csv'

df_grouped = pd.read_csv(file_path)

# df_grouped

### 필요한 함수 임폴트
import os
import json
import pandas
import pickle
from dotenv import load_dotenv
import google.generativeai as generativeai
import pandas as pd
import random
import time
import re

### .env file 로드
load_dotenv('/content/drive/MyDrive/임베딩/GCP gemini API 활용/.env')

### gemini key 불러오기기
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
generativeai.configure(api_key=GOOGLE_API_KEY)

## API 키 확인

import google.generativeai as generativeai
print(os.getenv("GOOGLE_API_KEY"))  # 올바른 값이 나오는지 확인

### 감성 분석 및 근거가 되는 keyword 추출 함수 정의
def analyze_review(text_input):
    # prompt 생성
    prompt = f"""
    다음 텍스트는 건강기능식품에 관한 소비자 리뷰입니다. 제품 하나당 모든 리뷰가 '/'로 구분되어져 포함되어 있습니다.
    해당 리뷰의 내용에서 긍정인 내용, 부정인 내용, 그리고 중립인 내용을 분류를 하고, 각 감성별로 리뷰를 요약하여 JSON 형식으로 제시해 주세요.
    각 감성별 리뷰 요약은 맥락이 비슷한 것만 남기고 요약을 하여, 각 감성별 리뷰의 글자 수가 300자를 넘지 않도록 요약해 주세요.
    그리고 리뷰 요약할 때 없는 내용을 창조하면 안되고, 리뷰 내용에 100% 충실하게 요약해야 해야 합니다.
    JSON 형식만 출력하세요. 아래 형식 외의 설명 문구는 절대 쓰지 마세요.

    텍스트: {text_input}

    출력 형식 예시:
    {{
      "정확도": 85,
      "감성별 리뷰 요약": {{
        "긍정": ["긍정 리뷰 요약"],
        "부정": ["부정 리뷰 요약"],
        "중립": ["중립 리뷰 요약"]
      }}
    }}
    """


    try:
        # 텍스트 분석 결과 생성
        model = generativeai.GenerativeModel("gemini-2.5-pro")

        # generation_config 객체를 생성하여 temperature를 설정합니다.
        # temperature 값은 0.0 (가장 보수적)에서 1.0 (가장 창의적) 사이로 조절할 수 있습니다.
        generation_config = generativeai.types.GenerationConfig(
            temperature=0.0,  # 원하는 temperature 값으로 변경하세요 (예: 0.2, 0.5, 0.9 등)
            top_p=0.9,
            top_k=50
        )

        response = model.generate_content(
            contents=[prompt],
            generation_config=generation_config # 여기에 generation_config를 전달합니다.
        )


        result_str = response.text
        if result_str:
            print(f'결과 : {result_str}')

            # JSON 부분만 추출
            match = re.search(r"\{[\s\S]*\}", result_str)
            if match:
                json_text = match.group(0).strip()
                try:
                    parsed_json = json.loads(json_text)
                    return parsed_json
                except json.JSONDecodeError:
                    print("JSON 디코딩 실패. 추출된 내용:", json_text)
                    return None
            else:
                print("JSON 패턴을 찾지 못했습니다.")
                return None


    except Exception as e:
        print(f'API 호출 오류 : {e}')
        return None



from tqdm.notebook import tqdm  # Colab/Jupyter용 진행바

### 전체 텍스트를 처리하는 함수 정의 (tqdm 적용)
def process_multiple_texts(text_list):
    results = {}
    for i, text_input in enumerate(tqdm(text_list, desc="리뷰 처리 진행률")):
        retry_count = 0
        max_retries = 5  # 최대 재시도 횟수
        wait_time = 20  # 초기 대기 시간 (초)

        while retry_count < max_retries:
            print(f"텍스트 {i+1} 처리 시도 {retry_count + 1}...")
            result = analyze_review(text_input)
            if result:
            #     results[f"텍스트 {i+1}"] = result
            #     break  # 성공 시 루프 종료
            # else:
            #     retry_count += 1
            #     wait_time = wait_time * 2 + random.uniform(0, 1)  # 지수 백오프 + 약간의 임의 시간 추가
            #     print(f"API 요청 실패. {wait_time:.2f}초 후 재시도...")
            #     time.sleep(wait_time)

                result = analyze_review(text_input)

                # 성공 조건을 "올바른 딕셔너리(JSON 파싱 성공)"로 한정
                if isinstance(result, dict):
                    results[f"텍스트 {i+1}"] = result
                    break  # 성공 시 루프 종료
                else:
                    retry_count += 1
                    wait_time = wait_time * 2 + random.uniform(0, 1)
                    print(f"API 요청 또는 JSON 파싱 실패. {wait_time:.2f}초 후 재시도...")
                    time.sleep(wait_time)

        else:  # 최대 재시도 횟수 초과 시
            results[f"텍스트 {i+1}"] = "감정 키워드 추출 실패 (최대 재시도 횟수 초과)"

    return results




# 여러 텍스트 처리
all_results = process_multiple_texts(review_text)

# 최종 결과 출력
# print(all_results)

# 최종 결과를 dict 자료 구조로 변환
with open('review_analysis.pkl', 'wb') as fw:
    pickle.dump(all_results, fw)

# 저장된 결과를 다시 불러오기
with open('review_analysis.pkl', 'rb') as fr:
    loaded_data = pickle.load(fr)

print(f'리뷰 데이터 분석의 결과 : \n{loaded_data}')

# 청킹 및 임베딩

In [ ]:
# 필요한 라이브러리 설치
!pip install pandas sentence_transformers --upgrade llama-index tqdm nltk llama-index-llms-google-genai llama-index-embeddings-huggingface

# 필요한 라이브러리 임폴트
import pandas as pd
from sentence_transformers import SentenceTransformer
from llama_index.core.node_parser import SentenceSplitter # from llama_index.core.node_parser.text.sentence import SentenceSplitter와 같음
from tqdm import tqdm
import ast

# 데이터 프레임 불러오기
df = pd.read_csv('/content/drive/MyDrive/임베딩/GCP gemini API 활용/6.리뷰분석(감성_리뷰만 있는 버전)')

# 1. CSV 불러오기 (리뷰 텍스트가 '리뷰' 컬럼에 있다고 가정)
df = pd.read_csv('/content/drive/MyDrive/임베딩/GCP gemini API 활용/6.리뷰분석(감성_리뷰만 있는 버전)')

# 2. 문장 단위 청킹 함수 정의
def chunk_text(text, chunk_size=200, chunk_overlap=50):
    splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_text(text)
    return chunks

# 3. 임베딩 모델 로드 (무료 다국어 모델)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 4. 결과 저장용 리스트 준비
rows = []

# 5. 전체 리뷰에 대해 처리 (청킹 + 임베딩)
for idx, row in tqdm(df.iterrows(), total=len(df)):
    고유번호 = row['고유번호']
    제품명 = row['제품명']
    카테고리 = row['카테고리']
    상세정보 = row['상세정보']
    긍정리뷰 = row['긍정_리뷰']
    부정리뷰 = row['부정_리뷰']
    중립리뷰 = row['중립_리뷰']

    # 결측치나 숫자 문제 해결: 빈 문자열로 채우고 문자열 변환
    if pd.isna(긍정리뷰):
        긍정리뷰 = ''
    긍정리뷰 = str(긍정리뷰)

    if 긍정리뷰.strip() == '':
        continue  # 빈 텍스트면 처리하지 않음

    chunks = chunk_text(긍정리뷰)
    for chunk in chunks:
        embedding = model.encode(chunk).tolist()
        rows.append({
            '고유번호': 고유번호,
            '제품명' : 제품명,
            '카테고리' : 카테고리,
            '상세정보' : 상세정보,
            '긍정리뷰' : 긍정리뷰,
            '부정리뷰' : 부정리뷰,
            '중립리뷰' : 중립리뷰,
            '긍정_청크': chunk,
            '긍정리뷰_임베딩': embedding
        })

# 6. DataFrame으로 변환
result_df = pd.DataFrame(rows)

# 7. 임베딩 벡터를 문자열로 저장하기 위해 변환
result_df['긍정리뷰_임베딩'] = result_df['긍정리뷰_임베딩'].apply(lambda x: str(x))

# 8. CSV로 저장
result_df.to_csv('/content/drive/MyDrive/임베딩/GCP gemini API 활용/reviews_chunked_embeddings4.csv', index=False)

print("처리 완료! '/content/drive/MyDrive/임베딩/GCP gemini API 활용/reviews_chunked_embeddings4.csv' 파일에 저장되었습니다.")

# 코사인 유사도 계산

In [ ]:
import pandas as pd
import ast

file_path4 = '/content/drive/MyDrive/임베딩/GCP gemini API 활용/reviews_chunked_embeddings4.csv'

df = pd.read_csv(file_path4)

# 문자열 컬럼 -> 리스트 변환
df['긍정리뷰_임베딩_int'] = df['긍정리뷰_임베딩'].apply(ast.literal_eval)

print(type(df.loc[0, '긍정리뷰_임베딩_int']))  # <class 'list'>

condition_zinc = (df.loc[:,'카테고리']=='아연')
df_zinc = df.loc[condition_zinc,:]

condition_vitamin = (df.loc[:,'카테고리']=='비타민')
df_vitamin = df.loc[condition_vitamin,:]

condition_probiotics = (df.loc[:,'카테고리']=='유산균')
df_probiotics = df.loc[condition_probiotics,:]

condition_omega3 = (df.loc[:,'카테고리']=='오메가3')
df_omega3 = df.loc[condition_omega3,:]

condition_protein = (df.loc[:,'카테고리']=='단백질')
df_protein = df.loc[condition_protein,:]

### 코사인 유사도 계산
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 카테고리 선택
category = input("원하는 건강기능식품 종류를 입력해주세요.\n\n 건강기능식품 종류: 아연, 비타민, 유산균, 오메가3, 단백질 中 \t")

if category == '아연':
    df = df_zinc
elif category == '비타민':
    df = df_vitamin
elif category == '유산균':
    df = df_probiotics
elif category == '오메가3':
    df = df_omega3
elif category == '단백질':
    df = df_protein

# iloc과 loc 같게 만들기
df = df.reset_index(drop=True)


# 3. 검색할 쿼리
# 하루 필요한 비타민과 미네랄이 모두 들어있는 제품
query = input("사용자 프롬프트:\t") # 테스트용 문장(바꿔서 해보면 됨)

# 4. 쿼리 임베딩 (리뷰 임베딩 모델과 동일한 모델 사용)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
query_embedding = model.encode([query])

# 5. 코사인 유사도 계산
embeddings_matrix = np.array(df['긍정리뷰_임베딩_int'].tolist())
similarities = cosine_similarity(query_embedding, embeddings_matrix)[0]

# 6. 상위 2개 문장 출력
top_n = 2
top_indices = similarities.argsort()[-top_n:][::-1]
unique_results = []
seen_ids = set()

for idx in top_indices:
    product_id = df.iloc[idx]['고유번호']

    if product_id not in seen_ids:
        seen_ids.add(product_id)
        unique_results.append((idx, similarities[idx]))

    if len(unique_results) == top_n:
        break

# 출력
def result(unique_results):
    outputs = []
    for idx, score in unique_results:
        row = df.loc[idx]
        outputs.append(
            f"\n제품명: {row['제품명']} | 고유번호: {row['고유번호']} | 카테고리: {row['카테고리']} "
            f"| 상세정보: {row['상세정보']} "
            f"\n긍정 리뷰: {row['긍정리뷰']} \n부정 리뷰: {row['부정리뷰']} "
            f"\n중립 리뷰: {row['중립리뷰']}"
        )
        print(f"\n인덱스: {idx} | 유사도: {score:.4f} | 제품명: {row['제품명']} | 고유번호: {row['고유번호']} | 카테고리: {row['카테고리']} "
              f"| 상세정보: {row['상세정보']} "
              f"\n긍정 리뷰: {row['긍정리뷰']}"
             # f"\n부정 리뷰: {row['부정리뷰']}\n중립 리뷰: {row['중립리뷰']}"
        )
    return outputs

product_result = result(unique_results)

# RAG 구현(LanghChain 기반)

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import pandas as pd
file_path4 = '/content/drive/MyDrive/임베딩/GCP gemini API 활용/reviews_chunked_embeddings4.csv'

df = pd.read_csv(file_path4)

In [ ]:
import ast
df['긍정리뷰_임베딩_int'] = df['긍정리뷰_임베딩'].apply(ast.literal_eval)

condition_zinc = (df.loc[:,'카테고리']=='아연')
df_zinc = df.loc[condition_zinc,:]

condition_vitamin = (df.loc[:,'카테고리']=='비타민')
df_vitamin = df.loc[condition_vitamin,:]

condition_probiotics = (df.loc[:,'카테고리']=='유산균')
df_probiotics = df.loc[condition_probiotics,:]

condition_omega3 = (df.loc[:,'카테고리']=='오메가3')
df_omega3 = df.loc[condition_omega3,:]

condition_protein = (df.loc[:,'카테고리']=='단백질')
df_protein = df.loc[condition_protein,:]

### PDF Load

In [ ]:
import glob

pdf_folder = "/content/drive/MyDrive/RAG/PDF/*.pdf"
pdf_files = glob.glob(pdf_folder)

In [ ]:
%%time
# 청크 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)

# PDF문서 로드 & 청크 분할
all_docs = []

for pdf_file in tqdm(pdf_files, desc = "PDF Load :"):
    loader = PyMuPDFLoader(pdf_file)
    docs = loader.load()
    all_docs.extend(docs)

all_chunks = text_splitter.split_documents(all_docs)

print(f"총 문서 수: {len(all_chunks)}")

In [ ]:
docs = []
for i, c in enumerate(all_chunks):
    docs.append({
        "id": f"doc-{i}",
        "title": c.metadata.get("source", ""),  # PDF 파일명
        "page": c.metadata.get("page", -1),    # 페이지 번호
        "text": c.page_content                 # 실제 텍스트
    })

### RAG

In [ ]:
import os, json, re, textwrap
from typing import List, Dict, Any, Tuple

from sentence_transformers import SentenceTransformer, CrossEncoder
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ==================================================
# 0. 데이터/카테고리 선택
# ==================================================
category = input("원하는 건강기능식품 종류를 입력해주세요.\n\n 건강기능식품 종류: 아연, 비타민, 유산균, 오메가3, 단백질 中 \t")

# 각 카테고리별 데이터프레임이 있다고 가정
if category == '아연':
    df = df_zinc
elif category == '비타민':
    df = df_vitamin
elif category == '유산균':
    df = df_probiotics
elif category == '오메가3':
    df = df_omega3
elif category == '단백질':
    df = df_protein
else:
    raise ValueError("지원하지 않는 카테고리입니다.")

df = df.reset_index(drop=True)

# ==================================================
# 1. 리뷰 기반 추천 (임베딩 + 코사인 유사도)
# ==================================================
embed_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def search_product_from_reviews(query: str, df: pd.DataFrame, top_n: int = 2):
    query_embedding = embed_model.encode([query])
    embeddings_matrix = np.array(df['긍정리뷰_임베딩_int'].tolist())
    similarities = cosine_similarity(query_embedding, embeddings_matrix)[0]

    top_indices = similarities.argsort()[-top_n:][::-1]
    unique_results = []
    seen_ids = set()

    for idx in top_indices:
        product_id = df.iloc[idx]['고유번호']
        if product_id not in seen_ids:
            seen_ids.add(product_id)
            unique_results.append((idx, similarities[idx]))
        if len(unique_results) == top_n:
            break

    return unique_results

def product_result(unique_results, df: pd.DataFrame):
    outputs = []
    for idx, score in unique_results:
        row = df.loc[idx]
        outputs.append(f"제품명: {row['제품명']}")
    return outputs

# ==================================================
# 2. 문서 기반 검색 (BM25 + Dense + RRF + Reranker)
# ==================================================
docs: List[Dict[str, Any]] = []  # TODO: 실제 문서 리스트 로드

EMBEDDING_MODEL = "BAAI/bge-m3"
RERANKER_MODEL = "BAAI/bge-reranker-v2-m3"
emb_model_doc = SentenceTransformer(EMBEDDING_MODEL)

def simple_tokenize_ko(text: str) -> List[str]:
    text = re.sub(r"[^0-9A-Za-z가-힣%·\.\-\s]", " ", text)
    return [t for t in text.split() if t]

bm25_corpus_tokens = [simple_tokenize_ko(d["text"]) for d in docs]
bm25 = BM25Okapi(bm25_corpus_tokens) if docs else None

try:
    import faiss
    use_faiss = True
except Exception:
    faiss = None
    use_faiss = False

if docs:
    doc_embeddings = emb_model_doc.encode([d["text"] for d in docs], batch_size=64, convert_to_numpy=True, show_progress_bar=True)
    if use_faiss:
        dim = doc_embeddings.shape[1]
        index = faiss.IndexFlatIP(dim)
        norms = np.linalg.norm(doc_embeddings, axis=1, keepdims=True) + 1e-12
        normed = doc_embeddings / norms
        index.add(normed.astype('float32'))
    else:
        index = None
else:
    doc_embeddings = np.zeros((0, 384), dtype='float32')
    index = None

def dense_search(query: str, top_k=40) -> List[Tuple[int, float]]:
    if not docs:
        return []
    q = emb_model_doc.encode([query], convert_to_numpy=True)[0]
    q = q / (np.linalg.norm(q) + 1e-12)
    if index is not None and use_faiss:
        D, I = index.search(q[np.newaxis, :].astype('float32'), top_k)
        return [(int(i), float(d)) for i, d in zip(I[0], D[0])]
    sims = (doc_embeddings @ q) / (np.linalg.norm(doc_embeddings, axis=1) + 1e-12)
    top_idx = np.argsort(-sims)[:top_k]
    return [(int(i), float(sims[i])) for i in top_idx]

def sparse_search(query: str, top_k=80) -> List[Tuple[int, float]]:
    if not docs or bm25 is None:
        return []
    tokens = simple_tokenize_ko(query)
    scores = bm25.get_scores(tokens)
    top_idx = np.argsort(-scores)[:top_k]
    return [(int(i), float(scores[i])) for i in top_idx]

def rrf_fuse(dense: List[Tuple[int, float]], sparse: List[Tuple[int, float]], k: int = 60, top_k: int = 50) -> List[int]:
    ranks: Dict[int, float] = {}
    for lst in [dense, sparse]:
        for rank, (idx, _) in enumerate(lst):
            ranks[idx] = ranks.get(idx, 0.0) + 1.0 / (k + rank + 1)
    fused = sorted(ranks.items(), key=lambda x: -x[1])[:top_k]
    return [idx for idx, _ in fused]

try:
    reranker = CrossEncoder(RERANKER_MODEL)
except Exception:
    reranker = None

def rerank(query: str, candidate_ids: List[int], top_k=10) -> List[int]:
    if not candidate_ids:
        return []
    pairs = [[query, docs[i]["text"]] for i in candidate_ids]
    if reranker:
        scores = reranker.predict(pairs)
    else:
        qset = set(simple_tokenize_ko(query))
        scores = [len(qset & set(simple_tokenize_ko(docs[i]["text"]))) for i in candidate_ids]
    order = np.argsort(-np.array(scores))[:top_k]
    return [candidate_ids[i] for i in order]

def build_context(query: str, max_chars: int = 2000, top_k_dense=40, top_k_sparse=40, top_k_final=8) -> Tuple[str, List[Dict[str, Any]]]:
    dense = dense_search(query, top_k=top_k_dense)
    sparse = sparse_search(query, top_k=top_k_sparse)
    fused_ids = rrf_fuse(dense, sparse, k=60, top_k=50)
    final_ids = rerank(query, fused_ids, top_k=top_k_final)

    selected = []
    total = 0
    for i in final_ids:
        d = docs[i]
        snippet = d["text"][:800]
        selected.append({"id": d["id"], "title": d.get("title", ""), "page": d.get("page", -1), "text": snippet})
        total += len(snippet)
        if total >= max_chars:
            break

    ctx_blocks = []
    for s in selected:
        header = f"[문서:{s['id']}] 제목:{s['title']} | 페이지:{s['page']}"
        ctx_blocks.append(header + "\n" + s["text"].strip())
    context_text = "\n\n".join(ctx_blocks)
    return context_text, selected

# ==================================================
# 3. LLM 프롬프트 & 파서
# ==================================================
SCHEMA_JSON = {
  "type": "object",
  "properties": {
    "사용자 입력": {"type": "string"},
    "추천 카테고리": {"type": "string"},
    "추천 제품": {"type": "string"},
    "전문가의 의견": {"type": "string"},
    "근거": {"type": "array", "items": {"type": "string"}},
    "참고 문서": {"type": "array"}
  },
  "required": ["사용자 입력", "추천 제품", "전문가의 의견"]
}

FEWSHOT = """
{
  "사용자 입력": "20대에게 추천하는 비타민",
  "추천 카테고리": "비타민",
  "추천 제품": "종합 비타민",
  "전문가의 의견": "20대는 균형 잡힌 영양 섭취가 필요하므로 종합 비타민이 적합합니다.",
  "근거": ["리뷰 기반 유사도 검색 결과", "제공 문서 참조"]
}

</예시>
위는 참고용입니다. 출력 시 예시 내용을 반복하지 말고, 실제 사용자 입력에 맞는 답변만 작성하세요.

엄격한 규칙:
- 반드시 위의 JSON 키만 사용. 추가 키/주석/머리말 금지.
- "전문가의 의견"은 제공된 문맥에서만 인용/요약. 모르면 "정보 부족"이라고 명시.
- 제공된 문맥 밖 정보를 추정하거나 발명 금지.
"""

SYSTEM_INSTRUCTIONS = f"""
당신은 영양학 전문가입니다.
주어진 리뷰 기반 추천 + 문서 기반 검색을 활용하여 JSON을 생성하세요.
절대 문맥 밖 지식으로 확장하지 마세요. 불충분하면 "정보 부족"을 사용하세요
JSON 스키마:
{json.dumps(SCHEMA_JSON, ensure_ascii=False)}
"""

MODEL_NAME = os.environ.get("HF_LLM", "skt/A.X-4.0-Light")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
textgen = pipeline("text-generation", model=model, tokenizer=tokenizer)

def robust_parse_json(text: str) -> Dict[str, Any]:
    m = re.search(r"\{[\s\S]*\}", text)
    if m:
        cand = m.group(0)
        try:
            return json.loads(cand)
        except Exception:
            pass
    return {"사용자 입력": "", "추천 제품": "", "전문가의 의견": "", "근거": []}

# ==================================================
# 4. 최종 질의 처리
# ==================================================
def answer_query(query: str, category: str, max_new_tokens: int = 256, temperature: float = 0.2) -> Dict[str, Any]:
    # 리뷰 기반 추천
    review_results = search_product_from_reviews(query, df)
    products = product_result(review_results, df)

    # 문서 기반 검색
    context_text, selected = build_context(query)

    prompt = f"""
    [시스템]
    {SYSTEM_INSTRUCTIONS}

    [문맥]
    {context_text}

    [리뷰 기반 추천]
    {products}

    [사용자 입력]
    {query}

    [카테고리]
    {category}

    [지시]
    {FEWSHOT}
    """

    out = textgen(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=temperature,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        return_full_text=False
    )[0]["generated_text"]

    data = robust_parse_json(out)
    if not data.get("추천 카테고리"):
        data["추천 카테고리"] = category
    if not data.get("참고 문서") and selected:
        data["참고 문서"] = [{"title": s["title"], "page": s["page"], "문서ID": s["id"]} for s in selected[:3]]
    return data

# ==================================================
# 5. 출력 렌더링
# ==================================================
def render_ko(data: Dict[str, Any]) -> str:
    parts = [
        f"사용자 입력 : {data.get('사용자 입력','')}",
        f"추천 카테고리 : {data.get('추천 카테고리','')}",
        f"추천 제품 : {data.get('추천 제품','')}",
        f"전문가의 의견 : {data.get('전문가의 의견','')}"
    ]
    if data.get("근거"):
        parts.append("근거 : " + "; ".join(map(str, data["근거"])))
    return "\n".join(parts)

# ==================================================
# 6. 실행부
# ==================================================
if __name__ == "__main__":
    query = input("사용자 질문: ")
    result = answer_query(query, category)
    print(render_ko(result))


# RAG 구현(LlamaIndex 기반)

In [ ]:
### 필요한 함수 임폴트
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import os
from dotenv import load_dotenv
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, Document
from llama_index.core import SimpleDirectoryReader
import unicodedata
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
import json

### SentenceSplitter

text_splitter = SentenceSplitter(
    chunk_size=200,
    chunk_overlap=50
)

### embedding

# 임베딩 모델 생성
embed_model = HuggingFaceEmbedding(
    model_name = 'paraphrase-multilingual-MiniLM-L12-v2',
    device='cpu' # gpu가 있다면 'cuda'
)

### llm

# google api key 불러오기
google_api_key = os.getenv("GOOGLE_API_KEY")
google_api_key = ''


# llm 생성
llm = GoogleGenAI(
    model='gemini-2.5-pro',
    request_timeout=120.0,
    temperature=0.0,
    api_key=google_api_key
)

### settings

# llm, embedding, text_splitter 모델 설정
Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

### index

import os
import unicodedata

# 직접 불러올 파일의 경로를 지정합니다.
file_path = "/content/drive/MyDrive/임베딩/GCP gemini API 활용/성분 효능.txt"


# 파일 경로가 존재하는지 확인합니다.
if not os.path.exists(file_path):
    raise ValueError(f"지정된 파일 '{file_path}'을(를) 찾을 수 없습니다.")

# 파일 경로를 리스트에 담아줍니다.
files = [file_path]

# 이제 'files' 리스트에는 지정된 txt 파일의 경로가 하나만 들어 있습니다.
# 이 리스트를 사용하여 문서를 로드하고 RAG를 구축하면 됩니다.
print(f"로드할 파일: {files}")

# 문서 로딩
txt_documents = SimpleDirectoryReader(input_files=files).load_data()

# document 자료 구조 --> 벡터 인덱스 생성: {'Node':임베딩 벡터,...}
'''
# VectorStoreIndex.from_documents(documents) 실행
# 내부적으로 문서 분할 -> 임베딩 -> 저장이 실행
'''

txt_index = VectorStoreIndex.from_documents(txt_documents)

### query_engine

# query_engine 생성
txt_engine = txt_index.as_query_engine(similarity_top_k=10, include_metadata=True)

### queryenginetool

## 도구 생성

# pdf_tool 생성
txt_tool = QueryEngineTool(
        query_engine=txt_engine,
        metadata=ToolMetadata(
            name='txt_tool',
            description='건강기능식품에 들어간 원료의 효능을 알려주는 도구입니다.'
        )
    )

### agent

# 시스템 프롬프트 정의
react_system_prompt = f"""
당신은 건강기능식품 AI 비서입니다. txt_tool을 반드시 활용하세요.
"""

# ReActAgent 생성
healthcare_agent = ReActAgent(
    tools=[txt_tool],
    llm=Settings.llm,
    system_prompt=react_system_prompt,
    verbose=True
)


response = await healthcare_agent.run(user_msg=f"""당신은 소비자에게 건강기능식품 구매 시 꼭 알아야 할 중요한 정보를 전달해주는 전문가입니다.
아래에 두가지 제품이 제시되어 있습니다. {product_result}에 있는 '상세정보'와 txt_tool에 있는 원료의 효능 텍스트를 이용하여 두가지 제품에 대한 효능을 요약하세요.
효능의 근거를 찾기 위해 txt_tool을 사용하여 관련 txt 문서를 참조해야 합니다.
제품 정보에는 고유번호는 적지 마세요.
부정 리뷰와 중립 리뷰 내용을 기반으로 제품을 구매하기 전에 사용자가 '주의해야 할 점'을 세 가지로 요약하세요.
각 주의할 점은 짧은 문장으로 정리하고, '~가 있으니 주의하세요.'와 같은 형식으로 작성하세요.
근거를 찾을 수 없는 경우에는 '출처 없음'이라고 작성하세요.

두가지 제품 : {product_result}

출력은 반드시 아래 JSON 형식으로 작성해야 합니다:
[
    {{
      "제품명": "...",
      "제품의 효능": "...",
      "제품 정보": "...",
      "근거": "..." - txt_tool을 참조하세요,
      "긍정 리뷰": "...",
      "주의할 점": "...",
      "요약": "..."
    }},
    ...
  ]""")
print(str(response))

# 웹 페이지 구현

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import ast
import json
import os
import re
from sklearn.metrics.pairwise import cosine_similarity
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from dotenv import load_dotenv
from typing import Any, Dict, List, Optional, Union

# .env 파일 로드 (같은 폴더에 본인의 구글 api 주소를 넣은 .env 파일 생성 확인!)
load_dotenv()

# --- 1. 초기 설정 및 자원 로딩 함수 ---
def safe_parse_embedding(x):
    """결측치(NaN)나 비정상 문자열을 안전하게 파싱합니다."""
    try:
        return ast.literal_eval(x) if pd.notna(x) else []
    except Exception:
        return []

@st.cache_data(show_spinner=False)
def load_data():
    """CSV 파일을 로드하고 임베딩 데이터를 리스트로 변환합니다."""
    file_path = 'reviews_chunked_embeddings4.csv' # 파일도 같은 폴더에 있어야 함!
    if not os.path.exists(file_path):
        st.error(f"파일을 찾을 수 없습니다: {file_path}")
        st.stop()
    df = pd.read_csv(file_path)
    df['긍정리뷰_임베딩_int'] = df['긍정리뷰_임베딩'].apply(safe_parse_embedding)
    return df

@st.cache_resource(show_spinner=False)
def load_all_resources():
    """RAG 인덱스와 필요한 모든 자원을 한 번만 로드합니다."""
    with st.spinner("모델과 데이터를 로드하는 중..."):
        # 임베딩 모델
        embed_model = HuggingFaceEmbedding(
            model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
            device='cpu' # GPU가 있다면 'cuda'
        )

        # LLM
        google_api_key = os.getenv("GOOGLE_API_KEY")
        if not google_api_key:
            st.error("GOOGLE_API_KEY 환경 변수가 설정되지 않았습니다.")
            st.stop()
        llm = GoogleGenAI(
            model='gemini-2.5-pro',
            request_timeout=120.0,
            temperature=0.0,
            api_key=google_api_key
        )

        # 설정 등록
        Settings.llm = llm
        Settings.embed_model = embed_model
        Settings.text_splitter = SentenceSplitter(chunk_size=200, chunk_overlap=50)

        # RAG 데이터 로드 및 인덱싱
        txt_path = "성분 효능.txt" # 해당 텍스트 파일도 같은 폴더 상에 있는지 확인 바람!
        if not os.path.exists(txt_path):
            st.error(f"RAG에 필요한 파일을 찾을 수 없습니다: {txt_path}")
            st.stop()
        txt_documents = SimpleDirectoryReader(input_files=[txt_path]).load_data()
        txt_index = VectorStoreIndex.from_documents(txt_documents)
        txt_engine = txt_index.as_query_engine(similarity_top_k=10, include_metadata=True)

    return {
        "df_reviews": load_data(),
        "llm": llm,
        "embed_model": embed_model,
        "txt_engine": txt_engine
    }

# --- Helper functions ---
def extract_json_from_text(s: str) -> Optional[str]:
    """텍스트에서 JSON 문자열을 다단계로 추출합니다."""
    # 1) ```json``` 블록 추출
    m = re.search(r'```json\s*(\[.*\])\s*```', s, re.DOTALL)
    if m:
        return m.group(1)

    # 2) 전체에서 첫 '[' ~ 마지막 ']'까지 추출
    first = s.find('[')
    last = s.rfind(']')
    if first != -1 and last != -1 and last > first:
        return s[first:last+1]

    # 3) 라인 단위로 조합 시도 (배열 시작/끝 또는 객체 시작)
    lines = s.splitlines()
    json_lines = [line for line in lines if line.strip().startswith('[') or line.strip().endswith(']') or line.strip().startswith('{') or (line.strip().startswith('"') and line.strip().endswith(',')) or (line.strip().startswith('}') and line.strip().endswith(','))]

    if json_lines:
        combined_json = "\n".join(json_lines)
        if combined_json.strip():
            return combined_json.strip()

    return None

# --- 2. 코사인 유사도 및 RAG 로직 함수 ---
def calculate_recommendations(
    filtered_df: pd.DataFrame,
    user_query: str,
    selected_keywords: List[str],
    embed_model: HuggingFaceEmbedding,
    top_n: int = 2
) -> pd.DataFrame:
    """
    코사인 유사도와 RAG 로직을 기반으로 사용자 쿼리에 가장 적합한
    제품을 추천하는 함수.

    Args:
        filtered_df (pd.DataFrame): '긍정리뷰_임베딩_int' 컬럼을 포함한 데이터프레임.
        user_query (str): 사용자의 검색 쿼리.
        selected_keywords (List[str]): 사용자가 선택한 카테고리(키워드) 리스트.
        embed_model (HuggingFaceEmbedding): 미리 로드된 임베딩 모델 객체.
        top_n (int): 추천할 제품의 개수.

    Returns:
        pd.DataFrame: 추천 제품 정보가 담긴 DataFrame.
    """
    # 1. 인덱스 재설정 및 데이터 복사 (인덱스 에러 해결)
    df = filtered_df.reset_index(drop=True)

    if df.empty:
        return pd.DataFrame()

    # 2. 사용자 쿼리 임베딩
    combined_query = " ".join(selected_keywords) + " " + user_query
    query_embedding = embed_model.get_text_embedding(combined_query.strip())

    # 3. 코사인 유사도 계산
    embeddings_matrix = np.array(df['긍정리뷰_임베딩_int'].tolist())
    similarities = cosine_similarity([query_embedding], embeddings_matrix)[0]

    # 4. 각 제품의 가장 높은 유사도 점수를 찾아서 DataFrame에 추가
    df['유사도'] = similarities

    # 5. 제품별로 가장 높은 유사도를 가진 행을 선택
    idx = df.groupby('고유번호')['유사도'].idxmax()
    df_unique = df.loc[idx].sort_values(by='유사도', ascending=False)

    # 6. 상위 n개 제품만 선택
    recommended_df = df_unique.head(top_n).copy()

    # 7. 유사도 점수가 0인 경우 추천 제품 없음으로 처리
    if recommended_df.empty or recommended_df['유사도'].iloc[0] == 0.0:
        return pd.DataFrame()

    return recommended_df

def answer_query(txt_engine: Any, products_info_json: str) -> Optional[List[Dict[str, Any]]]:
    """
    QueryEngine을 직접 호출하여 전문가 의견을 생성합니다.
    """
    user_prompt = f"""
    당신은 소비자에게 건강기능식품 구매 시 꼭 알아야 할 중요한 정보를 전달해주는 전문가입니다.
    **반드시 당신에게 제공된 문서(성분 효능.txt)에 있는 정보만을 사용해야 합니다. 다른 정보는 절대 사용하지 마세요.**

    아래에 두가지 제품이 제시되어 있습니다. {products_info_json}에 있는 '상세정보'와 **제공된 문서(성분 효능.txt)에 있는 원료의 효능 텍스트를 이용하여** 두가지 제품에 대한 효능을 요약하세요.
    효능의 근거를 찾기 위해 제공된 문서를 사용해야 합니다.
    제품 정보에는 고유번호는 적지 마세요.

    **각 제품에 대한 긍정, 부정, 중립 리뷰의 실제 내용을 기반으로 각각을 요약해야 합니다.** 만약 리뷰 내용이 없다면 '정보 없음'이라고 명시하세요.

    부정 리뷰와 중립 리뷰 내용을 기반으로 제품을 구매하기 전에 사용자가 '주의해야 할 점'을 세 가지로 요약하세요.
    각 주의할 점은 짧은 문장으로 정리하고, '~가 있으니 주의하세요.'와 같은 형식으로 작성하세요.
    근거를 찾을 수 없는 경우에는 '출처 없음'이라고 작성하세요.

    두가지 제품 : {products_info_json}

    출력은 반드시 아래 JSON 형식으로 작성해야 합니다:
    [
        {{
          "제품명": "...",
          "제품의 효능": "...",
          "제품 정보": "...",
          "근거": "..." - 제공된 문서를 참조하세요,
          "긍정 리뷰": "...",
          "부정 리뷰": "...",
          "중립 리뷰": "...",
          "주의할 점": "...",
          "요약": "..."
        }},
        ...
      ]"""

    # 1. QueryEngine 직접 호출
    try:
        response = txt_engine.query(user_prompt)
        response_str = str(response)
    except Exception as e:
        st.error(f"쿼리 엔진 호출 중 오류가 발생했습니다: {e}")
        return None

    # 2. JSON 파싱
    json_str = extract_json_from_text(response_str)

    if json_str:
        try:
            return json.loads(json_str)
        except Exception:
            pass

    # 최종적으로 전체 문자열에서 파싱 시도
    try:
        return json.loads(response_str)
    except (json.JSONDecodeError, SyntaxError) as e:
        st.error("LLM 응답에서 JSON을 파싱하지 못했습니다.")
        st.json(response_str)
        return None

# --- 3. Streamlit 앱 메인 UI ---
def main():
    if "page" not in st.session_state:
        st.session_state.page = 1
        st.session_state.df_reviews = pd.DataFrame()
        st.session_state.selected_supplement = None
        st.session_state.selected_supplement_name = None
        st.session_state.selected_keywords = []
        st.session_state.page2_checkbox_states = {}
        st.session_state.filtered_data = pd.DataFrame()
        st.session_state.user_free_text = ""
        st.session_state.recommended_products = pd.DataFrame()
        st.session_state.supplementary_reviews = pd.DataFrame()
        st.session_state.rag_result = None

    # 모든 리소스 사전 로드
    resources = load_all_resources()
    st.session_state.df_reviews = resources.get("df_reviews")
    st.session_state.llm = resources.get("llm")
    st.session_state.embed_model = resources.get("embed_model")
    st.session_state.txt_engine = resources.get("txt_engine")

    # --- 페이지별 UI 구성 ---
    # Page 1
    if st.session_state.page == 1:
        st.markdown("<h1 style='text-align: center;'>💊 건강식품 추천 챗봇</h1>", unsafe_allow_html=True)
        st.markdown("<h5 style='text-align: center;'>리뷰를 기반으로 맞춤형 건강식품을 추천해드립니다.</h5>", unsafe_allow_html=True)
        st.progress(25)
        st.markdown("단계 **1/4** | 25% 완료")
        st.divider()
        st.markdown("<h3 style='text-align: center;'>어떤 건강식품을 찾으시나요?</h3>", unsafe_allow_html=True)

        supplements = {
            "단백질 쉐이크": {"description": "💪 근육 건강 및 운동 후 회복", "csv_name": "단백질"},
            "오메가3": {"description": "🐟 심혈관 건강 및 뇌 기능", "csv_name": "오메가3"},
            "종합비타민": {"description": "🌈 전반적인 영양 균형", "csv_name": "비타민"},
            "아연": {"description": "⚡️ 면역력 및 피부 건강", "csv_name": "아연"},
            "유산균": {"description": "🌿 장 건강 및 소화 기능", "csv_name": "유산균"}
        }

        for supplement_name, data_item in supplements.items():
            if st.button(f"**{supplement_name}**\n\n{data_item['description']}", use_container_width=True, key=supplement_name):
                st.session_state.selected_supplement = data_item['csv_name']
                st.session_state.selected_supplement_name = supplement_name
                st.success(f"**{supplement_name}**를 선택하셨습니다.")
                st.session_state.page = 2
                st.rerun()

        st.divider()
        col1, col2 = st.columns([0.89, 0.11])
        with col1:
            st.button("이전", disabled=True)
        with col2:
            if st.button("다음 >", disabled=not st.session_state.selected_supplement):
                if st.session_state.selected_supplement:
                    st.session_state.page = 2
                    st.rerun()
                else:
                    st.warning("먼저 카테고리를 선택해주세요.")

    # Page 2
    elif st.session_state.page == 2:
        df_reviews = st.session_state.df_reviews
        selected_category = st.session_state.get('selected_supplement', None)
        if selected_category and not df_reviews.empty:
            st.session_state.filtered_data = df_reviews[df_reviews['카테고리'] == selected_category].copy()
            if st.session_state.filtered_data.empty:
                st.warning("선택하신 카테고리에 대한 데이터가 없습니다. 다른 카테고리를 선택해주세요.")
                st.session_state.page = 1
                st.stop()

        st.markdown("<h1 style='text-align: center;'>💊 건강식품 추천 챗봇</h1>", unsafe_allow_html=True)
        st.markdown("<h5 style='text-align: center;'>리뷰를 기반으로 맞춤형 건강식품을 추천해드립니다.</h5>", unsafe_allow_html=True)
        st.progress(50)
        st.markdown("단계 **2/4** | 50% 완료")
        st.divider()
        st.markdown("<h3 style='text-align: center;'>주요 관심사를 선택해주세요 (복수 선택 가능)</h3>", unsafe_allow_html=True)

        supplement_keywords = {
            "단백질": ["근육 증가", "맛있는", "대용량", "목 넘김이 편한"],
            "오메가3": ["심혈관 건강", "관절 건강", "알약 크기", "생선 맛 안나는"],
            "비타민": ["피로 회복", "브랜드 신뢰성", "알약 크기", "고함량"],
            "아연": ["면역력 강화", "알약 크기", "하루 섭취량", "부작용 없는"],
            "유산균": ["장 건강", "소화 개선", "활력 증진", "배변 활동"],
        }
        selected_supplement = st.session_state.get('selected_supplement', None)
        keywords = supplement_keywords.get(selected_supplement, [])
        selected_keywords = []

        cols = st.columns(2)
        for i, keyword in enumerate(keywords):
            col_index = i % 2
            with cols[col_index]:
                if f"chkbox_{keyword}" not in st.session_state.page2_checkbox_states:
                    st.session_state.page2_checkbox_states[f"chkbox_{keyword}"] = False
                is_checked = st.checkbox(
                    label=keyword,
                    key=f"chkbox_{keyword}",
                    value=st.session_state.page2_checkbox_states[f"chkbox_{keyword}"]
                )
                if is_checked:
                    selected_keywords.append(keyword)

        st.session_state.selected_keywords = selected_keywords
        st.divider()
        col1, col2 = st.columns([0.89, 0.11])
        with col1:
            if st.button("이전"):
                st.session_state.page = 1
                st.rerun()
        with col2:
            if st.button("다음 >"):
                if st.session_state.selected_keywords:
                    st.session_state.page = 3
                    st.rerun()
                else:
                    st.warning("하나 이상의 관심사를 선택해주세요.")

    # Page 3
    elif st.session_state.page == 3:
        st.markdown("<h1 style='text-align: center;'>💊 건강식품 추천 챗봇</h1>", unsafe_allow_html=True)
        st.markdown("<h5 style='text-align: center;'>리뷰를 기반으로 맞춤형 건강식품을 추천해드립니다.</h5>", unsafe_allow_html=True)
        st.progress(75)
        st.markdown("단계 **3/4** | 75% 완료")
        st.divider()
        st.markdown(
            """
            <style>
            .stTextArea > label { display: none; }
            </style>
            """,
            unsafe_allow_html=True
        )
        st.markdown("<h3 style='text-align: center;'>추가 요청사항을 자유롭게 입력해주세요</h3>", unsafe_allow_html=True)
        st.session_state.user_free_text = st.text_area(
            label="질문 입력창",
            placeholder="예: 알레르기가 있어서 특정 성분은 피하고 싶어요,",
            height=150,
            value=st.session_state.user_free_text,
            key="user_input_text"
        )
        st.markdown("<small>선택사항입니다. 이 단계는 건너뛸 수 있습니다.</small>", unsafe_allow_html=True)
        st.divider()
        col1, col2 = st.columns([0.86, 0.14])
        with col1:
            if st.button("이전"):
                st.session_state.page = 2
                st.rerun()
        with col2:
            if st.button("추천 받기"):
                if st.session_state.filtered_data.empty:
                    st.warning("선택하신 카테고리에 대한 리뷰 데이터가 없습니다. 다른 카테고리를 선택해주세요.")
                    st.stop()

                # 사용자 쿼리 조합
                user_query = st.session_state.user_free_text.strip()

                if not st.session_state.selected_keywords and not user_query:
                    st.warning("관심사 또는 추가 요청사항을 입력해주세요.")
                    st.stop()

                with st.spinner("최적의 제품을 찾고 있습니다..."):
                    recommended_products = calculate_recommendations(
                        st.session_state.filtered_data,
                        user_query,
                        st.session_state.selected_keywords,
                        st.session_state.embed_model
                    )
                    st.session_state.recommended_products = recommended_products

                # --- 이 부분을 추가/수정해주세요! ---
                if recommended_products.empty:
                    st.warning("선택하신 조건에 맞는 추천 제품을 찾을 수 없습니다. 조건을 완화하거나 다른 키워드를 시도해보세요.")
                    st.session_state.recommended_products = recommended_products # 빈 DataFrame 저장
                    st.session_state.rag_result = None # RAG 결과도 초기화
                    st.session_state.page = 4
                    st.rerun()

                with st.spinner("전문가 의견을 생성 중..."):
                    products_info_list = []
                    for _, product_row in recommended_products.iterrows():
                        all_reviews = st.session_state.df_reviews[st.session_state.df_reviews['제품명'] == product_row['제품명']]
                        negative_reviews = ' '.join(all_reviews['부정리뷰'].dropna().astype(str).tolist())
                        neutral_reviews = ' '.join(all_reviews['중립리뷰'].dropna().astype(str).tolist())

                        products_info_list.append({
                            "제품명": product_row['제품명'],
                            "고유번호": product_row['고유번호'],
                            "상세정보": product_row['상세정보'],
                            "긍정리뷰": product_row['긍정리뷰'],
                            "부정리뷰": negative_reviews,
                            "중립리뷰": neutral_reviews,
                            "카테고리": product_row['카테고리']
                        })

                    rag_result = answer_query(
                        st.session_state.txt_engine,
                        json.dumps(products_info_list, ensure_ascii=False)
                    )

                    st.session_state.rag_result = rag_result
                    st.session_state.page = 4
                    st.rerun()

    # Page 4
    elif st.session_state.page == 4:
        st.markdown("<h1 style='text-align: center;'>건강식품 추천 결과</h1>", unsafe_allow_html=True)
        col1, col2 = st.columns([0.8, 0.2])

        with col2:
            if st.button("다시 검색", use_container_width=True):
                st.session_state.page = 1
                st.session_state.selected_supplement = None
                st.session_state.selected_supplement_name = None
                st.session_state.selected_keywords = []
                st.session_state.page2_checkbox_states = {}
                st.session_state.filtered_data = pd.DataFrame()
                st.session_state.user_free_text = ""
                st.session_state.recommended_products = pd.DataFrame()
                st.session_state.rag_result = None
                st.rerun()

        st.progress(100)
        st.markdown("단계 **4/4** | 100% 완료")
        st.divider()

        st.markdown("### 👨‍💼 사용자 정보")
        with st.container():
            st.markdown("<div style='border:1px solid #eee; padding:12px; border-radius:8px'>", unsafe_allow_html=True)
            st.markdown(f"**관심 제품:** {st.session_state.get('selected_supplement_name', '선택되지 않음')}")
            st.markdown(f"**관심사:** {', '.join(st.session_state.get('selected_keywords', []))}")
            if st.session_state.get('user_free_text'):
                st.markdown(f"**추가 요청사항:** {st.session_state.user_free_text}")
            st.markdown("</div>", unsafe_allow_html=True)

        st.markdown("### 🧑‍⚕️ 추천 제품 및 전문가 의견") # 제목 변경
        with st.container():
            st.markdown("<div style='border:1px solid #eee; padding:12px; border-radius:8px'>", unsafe_allow_html=True)
            rag_result = st.session_state.get('rag_result', None)
            if rag_result and isinstance(rag_result, list):
                for product_data in rag_result:
                    if '제품명' in product_data:
                        st.markdown(f"#### {product_data.get('제품명', '제품명 없음')}")
                        st.markdown(f"**제품의 효능:** {product_data.get('제품의 효능', '정보 없음')}")
                        st.markdown(f"**제품 정보:** {product_data.get('제품 정보', '정보 없음')}")
                        st.markdown(f"**근거:** {product_data.get('근거', '출처 없음')}")
                        st.markdown(f"**요약:** {product_data.get('요약', '정보 없음')}")
                        st.markdown("---")
                        st.markdown("##### 📝 참고할만한 리뷰들")
                        st.markdown(f"- **긍정 리뷰**: {product_data.get('긍정 리뷰', '정보 없음')}")
                        st.markdown(f"- **부정 리뷰**: {product_data.get('부정 리뷰', '정보 없음')}")
                        st.markdown(f"- **중립 리뷰**: {product_data.get('중립 리뷰', '정보 없음')}")
                        st.markdown(f"**주의할 점:**")
                        warnings = product_data.get('주의할 점', [])
                        if isinstance(warnings, list):
                            for warning in warnings:
                                st.markdown(f"- {warning}")
                        else:
                            st.markdown(f"- {warnings}")
                        st.divider()
            else:
                st.warning("선택하신 조건에 맞는 추천 제품을 찾을 수 없습니다.") # 추천 제품이 없을 경우 메시지
                st.markdown("💡 추천 근거: 수집된 15,000개 리뷰를 분석하여 사용자의 관심사와 라이프스타일에 가장 적합한 제품을 추천해드립니다.", unsafe_allow_html=True)
            st.markdown("</div>", unsafe_allow_html=True)

if __name__ == "__main__":
    main()